# Demo 

Code for demo

In [ ]:
import json
import urllib.request
import random
import pandas as pd

IPADDRESS = '20.69.144.180'
base_uri = f'http://{IPADDRESS}:8000'

dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")
X=dataset[['sepal_length','sepal_width','petal_length','petal_width']]

def score(X, tenant):
    # tenant = random.choices(["tenant1","tenant2", "tenant3","tenant4","tenant5","tenant6","tenant7", "tenant8","tenant9"])[0]
    len = random.randint(2,50)
    data = {"tenant":tenant, "data": X.head(len).to_numpy().tolist()}
    body = str.encode(json.dumps(data))
    url = f'{base_uri}/score'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def get_deploymentmap():
    url = f'{base_uri}/get_deploymentmap'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def get_dynamic_tenantmap():
    url = f'{base_uri}/get_dynamic_tenantmap'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))
        
def get_dedicated_tenantmap():
    url = f'{base_uri}/get_dedicated_tenantmap'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


def updated_dedicated_pool(mapping):
    body = str.encode(json.dumps({"mapping":mapping}))
    url = f'{base_uri}/update_dedicated_pool'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, body, headers)
    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


## Scenario

100 models in total

Due to limisted resouces (CPU and Memory), we decided to load 10 models

Let's check which models are loaded in Hot memory

In [ ]:
get_dynamic_tenantmap()

Test inference as `tenant1` user

In [ ]:
%%time
score(X, "tenant1")

Test inference as `tenant2` user

In [ ]:
%%time
score(X, "tenant2")

Test inference as `tenant45` user

In [ ]:
%%time
score(X, "tenant45")

Check models in memory (Hot)

In [ ]:
get_dynamic_tenantmap()

And run inference again as the last `tenant45` user

In [ ]:
%%time
score(X, "tenant45")

Need to pin some of users's model regardless of the frequency of inference

API to update dedicated pool mapping. Try this to assign a tenant to a dedicated deployment

In [ ]:
get_dedicated_tenantmap()

Test inference as exsiting tenant user in the deicated memory (Hot)

In [ ]:
%%time
score(X, "tenant9")

In [ ]:
mapping ={"tenant91":"deployment9","tenant92":"deployment10"}
updated_dedicated_pool(mapping)

Test inference as `tenant91` or `tenant92`

In [ ]:
%%time
score(X, "tenant91")

In [ ]:
%%time
score(X, "tenant92")

Check tenants in memory (Hot) again

In [ ]:
get_dedicated_tenantmap()